In [1]:
from math import sqrt
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

from QOptCraft.basis import get_photon_basis
from QOptCraft.state import PureState, MixedState
from QOptCraft.invariant import (
    can_transition,
    can_transition_basis,
    photon_invariant,
    photon_invariant_basis
)

Define the cost function to optimize and the restrictions to fullfil.

In [18]:
def invariant(x: NDArray) -> float:
    """Generate a constraint to make the invariant equal to zero.

    Args:
        x (NDArray): amplitudes of our quantum states.

    Returns:
        float: sum of square differences between in and out invariants
            for all the input and output states.
    """
    # STATE 00
    inv_in = photon_invariant(PureState([[0,0,1,1]], [1]))
    fock_list_00 = get_photon_basis(4, 2)
    fock_list_00.remove([0,0,1,1])
    x_00 = x[:9]
    AMP = sqrt(1 - x_00 @ x_00)
    PROB = AMP ** 2
    state_out = PureState([[0,0,1,1], *fock_list_00], [AMP, *x_00])
    inv_out = photon_invariant(state_out)
    invariant_constraint = (inv_in - inv_out)**2

    # STATE 01
    inv_in = photon_invariant(PureState([[0,1,1,1]], [1]))
    fock_list_01 = get_photon_basis(4, 3)
    fock_list_01.remove([1,0,1,1])
    fock_list_01.remove([0,1,1,1])  # only 1011 appears with ancilla 11
    x_01 = x[9: 26]
    state_out = PureState(
        [[0,1,1,1], fock_list_01[0], *fock_list_01[1:]],
        [AMP, sqrt(1 - PROB - x_01 @ x_01), *x_01]
    )
    inv_out = photon_invariant(state_out)
    invariant_constraint += (inv_in - inv_out)**2

    # STATE 10
    inv_in = photon_invariant(PureState([[1,0,1,1]], [1]))
    x_10 = x[26: 43]
    state_out = PureState(
        [[1,0,1,1], fock_list_01[0], *fock_list_01[1:]],
        [AMP, sqrt(1 - PROB - x_10 @ x_10), *x_10]
    )
    inv_out = photon_invariant(state_out)
    invariant_constraint += (inv_in - inv_out)**2

    # STATE 11
    inv_in = photon_invariant(PureState([[1,1,1,1]], [1]))
    fock_list_11 = get_photon_basis(4, 4)
    fock_list_11.remove([1,1,1,1])
    fock_list_11.remove([2,0,1,1])  # only 1111 appears with ancilla 11
    fock_list_11.remove([0,2,1,1])  # only 1111 appears with ancilla 11
    x_11 = x[43: 74]
    state_out = PureState(
        [[1,1,1,1], fock_list_11[0], *fock_list_11[1:]],
        [AMP, sqrt(1 - PROB - x_11 @ x_11), *x_11]
    )
    inv_out = photon_invariant(state_out)
    invariant_constraint += (inv_in - inv_out)**2
    
    return invariant_constraint 


def cost(x: NDArray) -> float:
    """Cost function to minimize. Its value is minus the probability
    of measuring the ancilla 11.

    Args:
        x (NDArray): amplitudes of our quantum states.

    Returns:
        float: minus the probability.
    """
    prob = 1 - x[:9] @ x[:9]
    print(f"{prob = }")
    print(f"Invariant = {invariant(x)}\n")
    return -prob

Now we define the constraints of our optimizer. Since the probabilities must be between 0 and 1:

In [19]:
constr_00 = NonlinearConstraint(lambda x: x[:9] @ x[:9], 0, 1)
constr_01 = NonlinearConstraint(lambda x: 1 - x[:9] @ x[:9] + x[9: 26] @ x[9: 26], 0, 1)
constr_10 = NonlinearConstraint(lambda x: 1 - x[:9] @ x[:9] + x[26: 43] @ x[26: 43], 0, 1)
constr_11 = NonlinearConstraint(lambda x: 1 - x[:9] @ x[:9] + x[43: 74] @ x[43: 74], 0, 1)
constr_invariant = NonlinearConstraint(invariant, -1e-6, 1e-6)

constraints = [constr_00, constr_01, constr_10, constr_11]

Initial guess of parameters

In [20]:
x_init = np.array([1 / sqrt(15)] * 9 + [1 / sqrt(30)] * 17 + [1 / sqrt(30)] * 17 + [1 / sqrt(60)] * 31)
(res := minimize(cost, x_init, method="SLSQP", constraints=constraints))

prob = 0.40000000000000013
Invariant = 11.525104067044474

prob = 0.3999999923050733
Invariant = 11.525104168223365

prob = 0.3999999923050733
Invariant = 11.525104168223365

prob = 0.3999999923050733
Invariant = 11.525104167845331

prob = 0.3999999923050733
Invariant = 11.525104167845331

prob = 0.3999999923050733
Invariant = 11.525104143239371

prob = 0.3999999923050733
Invariant = 11.525104167845331

prob = 0.3999999923050733
Invariant = 11.525104167845331

prob = 0.3999999923050733
Invariant = 11.525104153631215

prob = 0.3999999923050733
Invariant = 11.525104143239371

prob = 0.40000000000000013
Invariant = 11.525104129316084

prob = 0.40000000000000013
Invariant = 11.525104129316087

prob = 0.40000000000000013
Invariant = 11.525104067044474

prob = 0.40000000000000013
Invariant = 11.525104139312567

prob = 0.40000000000000013
Invariant = 11.525104139312567

prob = 0.40000000000000013
Invariant = 11.525104143758544

prob = 0.40000000000000013
Invariant = 11.525104143758544

prob =

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.9999998569319516
       x: [ 1.261e-04  1.261e-04 ...  6.305e-05  6.305e-05]
     nit: 11
     jac: [ 5.043e-04  5.043e-04 ...  0.000e+00  0.000e+00]
    nfev: 826
    njev: 11

### Matriz de Knill

In [ ]:
V = np.array(
    [[-1/3, -sqrt(2)/3, sqrt(2)/3, 2/3],
     [sqrt(2)/3, -1/3, -2/3, sqrt(2)/3],
     [-sqrt(3 + sqrt(6))/3, sqrt(3 - sqrt(6))/3, -sqrt((3 + sqrt(6))/2)/3, sqrt(1/6 - 1/(3*sqrt(6)))],
     [-sqrt(3 - sqrt(6))/3, -sqrt(3 + sqrt(6))/3, -sqrt(1/6 - 1/(3*sqrt(6))), -sqrt((3+sqrt(6))/2)/3]
    ]
)